In [ ]:
 ==========================================
# ✅ UNIFICADOR DE CSV COMPLETO Y DEPURADO
# ==========================================
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Ruta de tu carpeta con los CSV
carpeta = r"C:\Users\Fer\OneDrive\Escritorio\Modelado de Minería de Datos\Proyecto_3\supermercado"
archivos_csv = glob.glob(os.path.join(carpeta, ".csv")) + glob.glob(os.path.join(carpeta, ".CSV"))

print("📂 Archivos encontrados:")
for a in archivos_csv:
    print("   •", os.path.basename(a))

print(f"\nTotal de archivos detectados: {len(archivos_csv)}")

# 2️⃣ Crear carpeta para los gráficos
carpeta_graficos = os.path.join(carpeta, "Graficos")
os.makedirs(carpeta_graficos, exist_ok=True)

# 3️⃣ Crear Excel de salida
ruta_salida_excel = os.path.join(carpeta, "Datos_Combinados_Todos.xlsx")
writer = pd.ExcelWriter(ruta_salida_excel, engine="openpyxl")

# 4️⃣ Leer y procesar cada CSV
dataframes = []

for archivo in archivos_csv:
    nombre = os.path.basename(archivo).replace(".csv", "").replace(".CSV", "")
    print(f"\n📄 Procesando archivo: {nombre}")

    try:
        # Intentar leer con distintos separadores
        try:
            df = pd.read_csv(archivo, encoding="utf-8", sep=",")
        except:
            df = pd.read_csv(archivo, encoding="latin1", sep=";")
        
        print(f"   ✔ Filas: {len(df)} | Columnas: {len(df.columns)}")

        # Guardar en una hoja del Excel
        df.to_excel(writer, sheet_name=nombre[:31], index=False)
        dataframes.append(df)

        # --- 5️⃣ Gráficos automáticos ---
        # Histogramas de variables numéricas
        if not df.select_dtypes(include='number').empty:
            df.select_dtypes(include='number').hist(figsize=(8, 6))
            plt.suptitle(f"Histograma - {nombre}", fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(carpeta_graficos, f"{nombre}_histograma.png"))
            plt.close()

        # Boxplot
        if not df.select_dtypes(include='number').empty:
            plt.figure(figsize=(8, 5))
            sns.boxplot(data=df.select_dtypes(include='number'))
            plt.title(f"Boxplot - {nombre}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(carpeta_graficos, f"{nombre}_boxplot.png"))
            plt.close()

        # Barras (solo la primera columna categórica)
        for col in df.select_dtypes(include='object').columns[:1]:
            plt.figure(figsize=(6, 4))
            df[col].value_counts().head(10).plot(kind='bar', color='lightcoral')
            plt.title(f"Top categorías - {col} ({nombre})")
            plt.ylabel("Frecuencia")
            plt.tight_layout()
            plt.savefig(os.path.join(carpeta_graficos, f"{nombre}_{col}_barras.png"))
            plt.close()

    except Exception as e:
        print(f"❌ Error con {nombre}: {e}")

# 6️⃣ Combinar todos los datos
if dataframes:
    df_total = pd.concat(dataframes, ignore_index=True, sort=False)
    resumen = df_total.describe(include='all')
    resumen.to_excel(writer, sheet_name="Resumen_General")
    print("\n📘 Hoja 'Resumen_General' creada con éxito.")
else:
    print("\n⚠️ No se pudieron combinar los archivos.")

writer.close()

print("\n✅ PROCESO FINALIZADO ✅")
print(f"Excel combinado: {ruta_salida_excel}")
print(f"Gráficos guardados en: {carpeta_graficos}")